In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Medha Trust\\Desktop\\Auto\\Chest_Cancer_Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Medha Trust\\Desktop\\Auto\\Chest_Cancer_Classification'

In [55]:
# 4. Update the entity  

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augumentation: bool
    params_image_size: list

In [56]:
# 5. Update the configuration manager in src config

# Configuration manager for base model preparation
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories
import tensorflow as tf

class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH
    ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


In [57]:
# 6. Update the components

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class Training:
    def __init__(self,config:TrainingConfig):
        self.config=config

    def get_base_model(self):
        self.model=tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.2
        )

        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augumentation:
            train_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator=valid_datagenerator

        self.train_generator=train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path,model):
            model.save(path)

    def train(self):
        self.steps_per_epoch=self.train_generator.samples//self.train_generator.batch_size
        self.validation_steps=self.valid_generator.samples//self.valid_generator.batch_size

        self.model.fit(
             self.train_generator,
             epochs=self.config.params_epochs,
             steps_per_epoch=self.steps_per_epoch,
             validation_steps=self.validation_steps,
             validation_data=self.valid_generator
        )

        self.save_model(
             path=self.config.trained_model_path,
             model=self.model
        )

In [58]:
# 7. Update the pipeline

try:
    config=ConfigurationManager()
    training_config=config.get_training_config()
    training=Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e

[2024-05-27 18:51:48,820: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-27 18:51:48,823: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-27 18:51:48,827: INFO: common: Created directory at : artifacts]
[2024-05-27 18:51:48,827: INFO: common: Created directory at : artifacts\training]
Found 42 images belonging to 2 classes.
Found 176 images belonging to 2 classes.
22/22 [==============================] - 47s 2s/step - loss: 122.0726 - accuracy: 0.5568 - val_loss: 55.6094 - val_accuracy: 0.4750
